In [47]:
import numpy as np
import matplotlib.pyplot as plt

In [48]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
input_str = '<' + input_str + '>'
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
           'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', '\'', '>']
print(f'{letters=}')
# print(f'{input_str=}')

len(letters)=32
502
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', "'", '>']


In [49]:
# generate SDR

SDR_len = 128
sparsity = 16 # 4
max_overlap = 3 # 1

sparse_prob = sparsity / SDR_len

letter_SDRs = {}

for letter in letters:
    while True:
        rnd_SDR = np.random.choice([0, 1], SDR_len, True, [1 - sparse_prob, sparse_prob])

        if (rnd_SDR > 0).sum() != sparsity:
            continue
        
        is_over_max = False
        for SDR in letter_SDRs.values():
            if (SDR & rnd_SDR).sum() > max_overlap:
                is_over_max = True
                break
        
        if is_over_max:
            continue
            
        break
    
    # print(f'{letter}')
    letter_SDRs.setdefault(letter, rnd_SDR)

for letter, SDR in letter_SDRs.items():
    print(f'{letter} : {(SDR > 0).sum()} {SDR.tolist()}')

a : 16 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
b : 16 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c : 16 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

In [50]:
# text to SDRs
input_SDRs = []

for inp in input_str:
    input_SDRs.append(letter_SDRs[inp])

print(f'{len(input_SDRs)=}')

# for SDR in input_SDRs:
#     print(f'{SDR.tolist()}')

len(input_SDRs)=502


In [51]:
# init start column
column_size = 16

start_SDR = letter_SDRs['<']

start_I = np.zeros((start_SDR.size, column_size))

active_start_SDR_idx = np.where(start_SDR == 1)

for as_idx in active_start_SDR_idx[0]:
    rnd_active = np.random.randint(column_size)

    start_I[as_idx][rnd_active] = 1

# for col in np.rot90(start_I):
#     print(f'{col.astype(int).tolist()}')

In [64]:
I = np.zeros((SDR_len, column_size))
O = np.zeros((SDR_len, column_size))

W = np.random.rand(SDR_len, column_size, SDR_len * column_size)

I = start_I

for SDR in input_SDRs[:1]:

    # O = W * I.reshape(1, 1, SDR_len * column_size)
    # O = np.sum(O, axis=2)

    O = np.dot(W, I.reshape(SDR_len * column_size))

    # recur
    # I = O * (SDR == 1)[np.newaxis].T
# O.shape
for col in np.rot90(O):
    print(f'{col.astype(int).tolist()}')

[7, 8, 9, 9, 8, 8, 7, 7, 7, 6, 8, 6, 8, 9, 7, 7, 6, 7, 7, 8, 9, 5, 7, 7, 7, 9, 7, 7, 6, 6, 7, 8, 7, 5, 7, 7, 6, 6, 9, 8, 6, 6, 9, 7, 6, 9, 9, 7, 8, 7, 7, 7, 7, 10, 8, 7, 10, 8, 8, 8, 7, 6, 7, 7, 6, 7, 7, 8, 7, 7, 7, 7, 7, 7, 8, 7, 8, 6, 7, 9, 7, 7, 6, 8, 8, 8, 7, 6, 7, 8, 8, 10, 7, 8, 7, 7, 7, 6, 9, 6, 7, 8, 8, 9, 7, 8, 8, 7, 7, 7, 8, 9, 8, 8, 7, 6, 9, 8, 6, 8, 7, 7, 8, 6, 8, 7, 9, 7]
[8, 8, 8, 9, 9, 7, 8, 7, 8, 6, 9, 5, 7, 8, 6, 6, 7, 8, 7, 7, 8, 10, 8, 6, 9, 6, 8, 9, 8, 9, 7, 8, 7, 7, 7, 8, 9, 9, 6, 8, 7, 7, 8, 8, 7, 7, 8, 7, 9, 6, 9, 6, 9, 9, 7, 7, 9, 6, 7, 9, 7, 10, 8, 6, 7, 8, 8, 7, 7, 8, 7, 8, 9, 8, 6, 6, 7, 5, 7, 8, 6, 9, 7, 6, 6, 8, 8, 7, 7, 7, 8, 7, 9, 9, 7, 9, 6, 9, 7, 8, 7, 8, 6, 8, 7, 8, 6, 9, 8, 6, 8, 7, 8, 9, 5, 8, 8, 7, 7, 9, 8, 8, 8, 7, 8, 9, 7, 8]
[9, 8, 5, 7, 9, 7, 9, 6, 8, 8, 9, 7, 8, 8, 7, 8, 7, 8, 7, 10, 8, 7, 9, 8, 8, 8, 7, 10, 10, 10, 7, 8, 9, 7, 7, 9, 9, 5, 7, 7, 5, 9, 8, 9, 6, 8, 7, 8, 8, 9, 8, 7, 8, 7, 8, 9, 6, 8, 7, 9, 8, 8, 5, 8, 9, 9, 7, 7, 8, 8, 9, 7, 8, 6